In [1]:
url_base =' https://www.buyrentkenya.com/houses-for-sale'
headers = {'User-Agent': 'Chrome'}

In [2]:
import csv
import os
import time
import requests
from bs4 import BeautifulSoup

In [3]:
# get the data
data = requests.get(url).content

NameError: name 'url' is not defined

In [ ]:
# creating a soup object

soup = BeautifulSoup(data, 'html.parser')

In [ ]:
results = soup.find_all(divclass="flex flex-col justify-between px-5 py-4 md:w-3/5")

In [ ]:
houses_data = []
# Get the path to the Kaggle Notebooks working directory
working_directory = os.path.join('/kaggle/working')

# Save the data to a CSV file in the Kaggle Notebooks working directory
csv_file_path = os.path.join(working_directory, 'houses-for-sale.csv')
with open(csv_file_path, 'w', newline='', encoding='utf-8') as f:
    csv_writer = csv.DictWriter(f, fieldnames=['title', 'location', 'size', 'selling price'])
    csv_writer.writeheader()
    csv_writer.writerows(houses_data)


In [ ]:
for page in range(1, 125):
    url= url_base + str(page)
    response = requests.get(url, headers=headers) 
    soup = BeautifulSoup(response.content, 'html.parser') 
    listings = soup.find_all('div', class_='px-5') 
    for listing in listings: 
        title = listing.find('span', class_='hidden').text.strip() 
        location = listing.find('p', class_='text-sm').text.strip() 
        size = listing.find('a', class_='text-grey-500').text.strip() 
        price = listing.find('div', {'class': 'flex justify-between items-center'}).text.strip() 
        houses_data.append({'title': title, 'location': location, 'size': size, 'selling price': price})


In [ ]:
print(listings)